In [1]:
# import packages

import mxnet as mx

from mxnet import image
from mxnet import init
from mxnet import gluon
from mxnet import ndarray as nd
from tqdm import tqdm
from mxnet import autograd
from mxnet.gluon.model_zoo.vision import *
import numpy as np
import time
import h5py
import os
import gc
from time import time

# READ Features

In [16]:
def load_dataset(network_name, shape, data_scope, batch_size=32):
    features_file_path = "features/%s_%d_%s_features.h5" % (network_name,shape,data_scope)
    features = None
    with h5py.File(features_file_path) as f:
        features = np.array(f['features'])
    
    labels_file_path = "features/%d_%s_labels.h5" % (shape,data_scope)
    labels = None
    with h5py.File(labels_file_path) as f:
        labels = np.array(f['labels'])
    return gluon.data.DataLoader(gluon.data.ArrayDataset(nd.array(features), nd.array(labels)), batch_size)

In [3]:
ds = load_dataset('resnet18_v2',224,'valid')

In [4]:
for features, labels in ds:
    labels = labels.as_in_context(mx.gpu())
    features = features.as_in_context(mx.gpu())
    print(len(labels), type(labels), type(features))
    break

128 <class 'mxnet.ndarray.ndarray.NDArray'> <class 'mxnet.ndarray.ndarray.NDArray'>


In [5]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [6]:
def accuracy(output, labels):
    return nd.mean(nd.argmax(output, axis=1) == labels).asscalar()

In [7]:
def evaluate(net, data):
    loss, acc, n = 0., 0., 0.
    steps = len(data)
    for data, label in data:
        data, label = data.as_in_context(ctx), label.as_in_context(ctx)
        output = net(data)
        acc += accuracy(output, label)
        loss += nd.mean(softmax_cross_entropy(output, label)).asscalar()
    return loss/steps, acc/steps

In [8]:
def fit(net, train_data, valid_data, num_epochs=20, learning_rate=1e-4, wd=1e-5, print_batches=None, ctx=mx.gpu()):        
    net.collect_params().reset_ctx(ctx)
    net.hybridize()
    
    print("Start training on ", ctx)
    
    trainer = gluon.Trainer(net.collect_params(), 'adam', {'learning_rate': learning_rate, 'wd': wd})
            
    for epoch in range(num_epochs):
        train_loss, train_acc = .0, .0
        steps = len(train_data)
        for features, labels in train_data:
            features = features.as_in_context(ctx)
            labels = labels.as_in_context(ctx)
            with autograd.record():
                output = net(features)
                loss = softmax_cross_entropy(output, labels)
            batch_size =  len(labels)
            loss.backward()
            trainer.step(batch_size)

            train_loss += nd.mean(loss).asscalar()
            train_acc += accuracy(output, labels)
        val_loss, val_acc = evaluate(net, valid_data)
        print("Epoch %d. loss: %.4f, acc: %.2f%%, val_loss %.4f, val_acc %.2f%%" % 
              (epoch+1, train_loss/steps, train_acc/steps*100, val_loss, val_acc*100))

In [9]:
ctx = mx.gpu()

In [18]:
def get_network_classifier(ctx):
    num_outputs = 120
    net = gluon.nn.HybridSequential()
    with net.name_scope():
#         net.add(gluon.nn.Flatten())
        net.add(gluon.nn.Dense(256, activation='relu'))
        net.add(gluon.nn.Dropout(0.3))
        net.add(gluon.nn.Dense(num_outputs))
    net.initialize(ctx=ctx)
    return net

In [11]:
def train(classifier, pretrain_network_name, input_shape):
    train_data = load_dataset(pretrain_network_name, input_shape, 'train')
    valid_data = load_dataset(pretrain_network_name, input_shape, 'valid')
    print("train %s" % pretrain_network_name)
    fit(classifier,train_data,valid_data)    
    model_file_path = "models/%s_%d.params" % (pretrain_network_name, input_shape)
    classifier.save_params(model_file_path)

In [12]:
claasifier = get_network_classifier(ctx)

In [13]:
claasifier

HybridSequential(
  (0): Dense(256, Activation(relu))
  (1): Dropout(p = 0.5)
  (2): Dense(120, linear)
)

In [14]:
# input shape 为224 的网络模型
net_224_list = ['resnet152_v1',
                'densenet121','densenet161','densenet169','densenet201']
# input shape 为299 的网络模型
net_299_list = ['inception_v3']


In [19]:
for network_name in net_224_list:
    train(claasifier, network_name, 224)

train resnet152_v1
Start training on  gpu(0)
Epoch 1. loss: 4.7834, acc: 1.24%, val_loss 4.7850, val_acc 1.11%
Epoch 2. loss: 4.7831, acc: 1.24%, val_loss 4.7848, val_acc 1.11%
Epoch 3. loss: 4.7825, acc: 1.24%, val_loss 4.7846, val_acc 1.11%
Epoch 4. loss: 4.7820, acc: 1.24%, val_loss 4.7844, val_acc 1.11%
Epoch 5. loss: 4.7816, acc: 1.24%, val_loss 4.7842, val_acc 1.11%


KeyboardInterrupt: 